In [ ]:
import pandas as pd
import numpy as np
import project_functions as pf
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime

from transformers import (AutoTokenizer,
                          BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          get_linear_schedule_with_warmup)

from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

In [ ]:
class TextDataset(Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):

        return len(self.labels)

    def get_labels(self):

        return self.labels

# Defining focal loss to use during training
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        Args:
            alpha (tensor or float): tensor of shape (num_labels,) with a separate alpha for each label,
                                     or float for a constant alpha across labels.
            gamma (float): focusing parameter.
            reduction (str): 'none' | 'mean' | 'sum'.
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):

        if self.alpha is not None:
            self.alpha = self.alpha.to(inputs.device)


        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        focal_loss = (1 - pt) ** self.gamma * BCE_loss

        if self.alpha is not None:
            focal_loss = self.alpha * focal_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [ ]:
# loading data
train_val_data = pd.read_csv('train_val.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
# splitting reviews into sentences and whole reviews - train set
sentences_train = pf.transform_sentences(train_val_data)
texts_train = pf.transform_text(train_val_data)

In [ ]:
# splitting reviews into sentences and whole reviews - test set
sentences_test = pf.transform_sentences(test_data)
texts_test = pf.transform_text(test_data)

In [ ]:
# defining alpha weights as inverted frequency of labels, setting gamma parameter
freqs = sentences_train.iloc[:,1:].to_numpy().sum(axis=0)
num = len(sentences_train)

inv_freq = num / freqs

alpha_weights = torch.tensor(inv_freq)
gamma_val = torch.tensor(2.0)

In [ ]:
# defining model parameters and seed for comparison purposes
model_path = 'sdadas/polish-roberta-large-v2'
ModelClass = RobertaForSequenceClassification

use_gpu_if_available = True
batch_size_train = 8
batch_size_eval = 8
max_tokenizer_length = 512

num_epochs = 8
warming_steps = 100
lr = 2e-5
weight_decay = 0.01
eval_steps_per_epoch = 1

torch.manual_seed(123)

In [ ]:
# setting up device for training and testing
device = 'cuda' if torch.cuda.is_available() and use_gpu_if_available else 'cpu'
print(f'Training device set to: {device}')

In [ ]:
# transferring parameters to device
gamma_val.to(device)
alpha_weights.to(device)

In [ ]:
# defining the number of labels for the multilabel task
classes = sentences_train.columns[1:]
labels = sentences_train.columns[1:]
labels = [s.strip() for s in labels]

NUM_LABELS = len(labels)
id2label = {idx: label for idx, label in enumerate(sorted(labels))}
label2id = {label: idx for idx, label in enumerate(sorted(labels))}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, max_length=max_tokenizer_length)
model = ModelClass.from_pretrained(model_path, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)
model.train()

In [ ]:
# preparing data for training 
train_texts = list(sentences_train['text'])
test_texts = list(sentences_test['text'])

train_labels = sentences_train.iloc[:,1:].astype(int).to_numpy()
test_labels = sentences_test.iloc[:,1:].astype(int).to_numpy()

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_tokenizer_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_tokenizer_length)

In [ ]:
train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size_eval, shuffle=False, pin_memory=True)

In [ ]:
# model setup
eval_step_list = np.linspace(0, len(train_dataloader), eval_steps_per_epoch+1).astype(int).tolist()[1:]

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warming_steps, num_training_steps=total_steps)
criterion = FocalLoss(alpha=alpha_weights, gamma=gamma_val, reduction='mean')

In [ ]:
# Training loop
for epoch in range(num_epochs):
    total_train_loss = 0.0
    loss_step_counter = 0

    for step, batch in enumerate(tqdm(train_dataloader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        labels = labels.float()

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_train_loss += loss.item()
        loss_step_counter += 1
        
        if (step + 1) in eval_step_list:
            eval_time = time.perf_counter()


            eval_time = time.perf_counter() - eval_time
            print(f'Epoch [{epoch + 1}/{num_epochs}], '
                  f'Step [{step + 1}/{len(train_dataloader)}], '
                  f'Train Loss: {total_train_loss / loss_step_counter:.2f}, '
                  f'lr: {optimizer.param_groups[0]["lr"]:.3e}, '
                  f'Eval Time: {eval_time:.2f}')

            total_train_loss = 0.0
            loss_step_counter = 0

# saving the model
# model.save_pretrained(f'{path}roberta_focal_{epoch}')
# tokenizer.save_pretrained(f'{path}roberta_focal_{epoch}')

In [ ]:
# getting raw data from model
labels_test, predictions_test = pf.get_test_data(model, test_dataloader, device)

In [ ]:
# splitting predictions based on predefined split value
split_val = 0.35

labels_test = np.array(labels_test)
predictions_test = np.array(predictions_test)
preds_test = (predictions_test > split_val).astype(int)

In [ ]:
# final evaluation
test_dict = pf.get_test_evaluation(labels_test, preds_test, classes)

### Text model

In [ ]:
# reusing sentence model and setting new parameters
model_path = focal_model_path
ModelClass = RobertaForSequenceClassification

use_gpu_if_available = True
batch_size_train = 2
batch_size_eval = 2
max_tokenizer_length = 512

num_epochs = 4
warming_steps = 100
lr = 2e-5
weight_decay = 0.01
eval_steps_per_epoch = 1 

torch.manual_seed(123)

In [ ]:
# setting up device for training and testing
device = 'cuda' if torch.cuda.is_available() and use_gpu_if_available else 'cpu'
print(f'Training device set to: {device}')

In [ ]:
# defining the number of labels for the multilabel task
classes = sentences_train.columns[1:]
labels = sentences_train.columns[1:]
labels = [s.strip() for s in labels]

NUM_LABELS = len(labels)
id2label = {idx: label for idx, label in enumerate(sorted(labels))}
label2id = {label: idx for idx, label in enumerate(sorted(labels))}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, max_length=max_tokenizer_length)
model = ModelClass.from_pretrained(model_path, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)
model.train()

In [ ]:
# preparing data for training 
train_texts = list(texts_train['text'])
test_texts = list(texts_test['text'])

train_labels = texts_train.iloc[:,1:].astype(int).to_numpy()
test_labels = texts_test.iloc[:,1:].astype(int).to_numpy()

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_tokenizer_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_tokenizer_length)

In [ ]:
train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size_eval, shuffle=False, pin_memory=True)

In [ ]:
# model setup
eval_step_list = np.linspace(0, len(train_dataloader), eval_steps_per_epoch+1).astype(int).tolist()[1:]
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warming_steps, num_training_steps=total_steps)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
# Training loop
for epoch in range(num_epochs):
    total_train_loss = 0.0
    loss_step_counter = 0

    for step, batch in enumerate(tqdm(train_dataloader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        labels = labels.float()

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_train_loss += loss.item()
        loss_step_counter += 1

        if (step + 1) in eval_step_list:
            eval_time = time.perf_counter()

            
            eval_time = time.perf_counter() - eval_time
            print(f'Epoch [{epoch + 1}/{num_epochs}], '
                  f'Step [{step + 1}/{len(train_dataloader)}], '
                  f'Train Loss: {total_train_loss / loss_step_counter:.2f}, '
                  f'lr: {optimizer.param_groups[0]["lr"]:.3e}, '
                  f'Eval Time: {eval_time:.2f}')

            total_train_loss = 0.0
            loss_step_counter = 0

# saving the model
# model.save_pretrained(f'{path}roberta_focal_text_{epoch}')
# tokenizer.save_pretrained(f'{path}roberta_focal_text_{epoch}')

In [ ]:
# getting raw data from model
labels_test, predictions_test = pf.get_test_data(model, test_dataloader, device)

In [ ]:
# splitting predictions based on predefined split value
split_val = 0.20

labels_test = np.array(labels_test)
predictions_test = np.array(predictions_test)
preds_test = (predictions_test > split_val).astype(int)

In [ ]:
# final evaluation
test_dict = pf.get_test_evaluation(labels_test, preds_test, classes)